In [ ]:
import torch
import torch.nn as nn
import pandas as pd
import warnings
import os
warnings.filterwarnings('ignore')
parent_path = 'C:\\Users\\ismai\\Documents\\Comment-Section-UI\\angular-interactive-comments-section\\flask\\data'

In [ ]:

file_name = 'fox-new-york-could-face'
file = os.path.join(parent_path, f'{file_name}.csv')
df = pd.read_csv(file)

In [ ]:
df.head(5)

In [ ]:
list(df.iloc[1][1:])

In [ ]:
def get_so_far_comments(ith_comment):
    # row_index = df.loc[df['id'] == comment_id].index[0]
    ids = df.iloc[0:ith_comment+1]['id']
    return list(ids)


In [ ]:
# get_so_far_comments(1)

In [ ]:
df_trend = pd.read_csv(os.path.join(parent_path, f'{file_name}-trends.csv'))

In [ ]:
df_trend.shape

In [ ]:
def get_single_trend_score(trend):
    print("label: ", trend)
    return list(df_trend[trend])

In [ ]:
# get_single_trend_score('toxic')

In [ ]:
previous_emotion = []

In [ ]:
import random

def get_emotion(escores, id):
    # print("id")
    if id == 0:
        previous_emotion = escores

    if id%100 == 0:
        X = escores[27]*0.6
        escores[27] = escores[27] - X
        y = X*(random.sample(range(0, 100), 1)[0]/100)
        random_numbers = random.sample(range(0, 27), 3)
        escores[random_numbers[0]] = X*0.2
        escores[random_numbers[1]] = X*0.7
        escores[random_numbers[2]] = X*0.1
        previous_emotion = escores
        return escores
    
    if id%200 == 0:
        X = escores[27]*0.6
        escores[27] = escores[27] - X
        y = X*(random.sample(range(0, 100), 1)[0]/100)
        random_numbers = random.sample(range(0, 27), 3)
        escores[random_numbers[0]] = X*0.1
        escores[random_numbers[1]] = X*0.6
        escores[random_numbers[2]] = X*0.3
        previous_emotion = escores
        return escores
    
    return previous_emotion


In [ ]:
from score_aggregation import get_emotion_trends_polarity

def getData(index):
    ids = get_so_far_comments(ith_comment=index)
    print("Index: ", index)
    print("So far Comments: ", ids)
    escores, tscores, polarity = get_emotion_trends_polarity(ids)
    return escores, tscores, polarity

In [22]:
from flask import Flask, request, jsonify
from flask_cors import CORS
import json

app = Flask(__name__)
cors = CORS(app, resources={r"/api/*": {"origins": "*"}})

@app.route('/api/new-comment', methods=['POST'])
def welcome():
    comment = request.json['content']
    img = request.json['img']
    isYou = request.json['isYou']
    score = request.json['score']
    username = request.json['username']
    print("new: ", comment)
    return 'Successfully comment added!'

@app.route('/api/data/<int:id>', methods=['GET'])
def getApiData(id: int):
    escores, tscores, polarity = getData(id)
    escores = get_emotion(escores, id)
    return json.dumps({"escores": escores, "tscores": tscores, "polarity": polarity})

@app.route('/api/data/single-trend/<string:label>', methods=['GET'])
def getSingleTrend(label: str):
    return json.dumps({"singletrend": get_single_trend_score(label)})

if __name__ == '__main__':
    app.run(host='127.0.0.1', port=5000)